# 建立初始速度预测场

In [ ]:
from phi.torch.flow import *
import tqdm

device = torch.device('cuda')
BOUNDS = Box(x=1, y=1)
RESOLUTION = 100


@jit_compile
def step(v, s, dt, inflow):
    s = advect.mac_cormack(s, v, dt) + 0.2 * resample(inflow, to=s, soft=True)
    buoyancy = resample(s * (0, 0.1), to=v)
    v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, _ = fluid.make_incompressible(v, (), Solve('auto', 1e-3))
    return v, s


v0 = StaggeredGrid(
    values=0,
    extrapolation=extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=spatial(x=RESOLUTION, y=RESOLUTION),
)
smoke0 = CenteredGrid(
    values=0,
    extrapolation=extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=spatial(x=RESOLUTION, y=RESOLUTION),
)
inflow0 = Sphere(x=0.5, y=0.15, radius=0.1)

vel, smoke = v0, smoke0
vel_list = []
smoke_list = []
for _ in tqdm.trange(100):
    vel, smoke = step(vel, smoke, 0.2, inflow0)
    vel_list.append(vel)
    smoke_list.append(smoke)